In [ ]:
!apt-get install gcc-4.8 gcc-5 g++-4.8 g++-5
!apt-get install libblas-dev liblapack-dev
!apt-get install libopenblas-dev
!apt-get install gfortran-4.8
!apt-get install gfortran-5
!apt-get install scons
!apt-get install libpng-dev
!apt-get install libboost-all-dev freeglut3-dev
!apt-get install xvfb
!apt-get install libspatialindex-dev

In [ ]:
!pip install torch==1.3.0 rtree trimesh

In [ ]:
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 10
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-4.8 10
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20

In [ ]:
!git clone https://github.com/mszarski/diffsim.git

In [10]:
!chmod +x diffsim/arcsim/dependencies/taucs/configure

In [ ]:
%cd /content

In [ ]:
%cd diffsim
!update-alternatives --set gcc "/usr/bin/gcc-4.8"
!update-alternatives --set g++ "/usr/bin/g++-4.8"
%cd arcsim/dependencies/
!make 
%cd ../..
!update-alternatives --set gcc "/usr/bin/gcc-5"
!update-alternatives --set g++ "/usr/bin/g++-5"
!make -j 8
%cd pysim
!ln -s ../arcsim/conf ./conf
!ln -s ../arcsim/materials ./materials
!ln -s ../arcsim/meshes ./meshes


In [ ]:
##restart runtime here

In [1]:
%cd /content/diffsim/pysim

/content/diffsim/pysim


In [2]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/draping_workspace'       # path to your project on Drive

drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%writefile conf/draping.json
{
    "frame_time": 0.125,
    "frame_steps": 1,
    "end_time": 10,
    "cloths": [
    {
        "mesh": "/content/drive/My Drive/draping_workspace/2d_shape_flatmesh_99pc.obj",
        "transform":{"scale":0.01,"translate":[-0.1,-0.3,0.3]},
        "materials": [{"data": "materials/gray-interlock.json",
                       "thicken": 2}],
        "remeshing": {
            "refine_angle": 0.3,
            "refine_compression": 0.005,
            "refine_velocity": 0.5,
            "size": [10e-3, 100e-3],
            "aspect_min": 0.2
        }
    }
    ],
    "obstacles": [
    {
        "mesh": "/content/drive/My Drive/draping_workspace/3d_shape.obj",
        "transform":{"scale":0.01,"translate":[-0.1,-0.3,-0.5]},
        "velocity": [0, 0, 0, 0, 0, 0],
        "movable": 0
    }
    ],
    "disable":["remeshing", "proximity"],
    "handles": [{"nodes": [302, 198, 2, 23]}],
    "gravity":[0, 0, -2],
    "magic": {"repulsion_thickness": 5e-3, "collision_stiffness": 1e6}
}

Writing conf/draping.json


In [10]:
import trimesh
import trimesh.transformations as tf
import numpy as np

curved_mesh = trimesh.load('/content/drive/My Drive/draping_workspace/3d_shape.obj',process=False)
flat_mesh = trimesh.load('/content/drive/My Drive/draping_workspace/2d_shape_flatmesh_99pc.obj',process=False)
scene = trimesh.Scene(base_frame='world')
curved_mesh.apply_scale(0.01)
flat_mesh.apply_scale(0.01)

transform = tf.translation_matrix([-0.1,-0.3,0.3])
flat_mesh.apply_transform(transform)
f_id = scene.add_geometry(flat_mesh)

transform = tf.translation_matrix([-0.1,-0.3,-0.5])
curved_mesh.apply_transform(transform)
c_id = scene.add_geometry(curved_mesh)

handles = [302, 198, 2, 23]
steps = 30

trajectories = []

for handle in handles:
    ray_origin = flat_mesh.vertices[handle]
    
    ray_direction = curved_mesh.vertices[handle] - flat_mesh.vertices[handle]
    ray_direction = ray_direction

    locations, index_ray, index_tri = curved_mesh.ray.intersects_location(
        ray_origins=np.array([ray_origin]),
        ray_directions=np.array([ray_direction]))

    location = locations[0]
    path = np.dstack([np.linspace(ray_origin[0],location[0],steps),np.linspace(ray_origin[1],location[1],steps),np.linspace(ray_origin[2],location[2],steps)])

    path = path.squeeze()
    trajectories.append(path)

trajectories = np.array(trajectories) #this is positions, not vs
trajectories = np.diff(trajectories,axis=1) #vs
trajectories = trajectories.transpose(1,0,2)
trajectories = np.concatenate([trajectories,np.expand_dims(np.zeros_like(trajectories[0]),0)])

In [13]:
!rm -rf /content/default_out

In [14]:
import torch
import arcsim
import gc
import time
import json
#import sysx
import gc
import os
#import numpy as np

from datetime import datetime
now = datetime.now()
timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

out_path = '/content/default_out'

if not os.path.exists(out_path):
  os.mkdir(out_path)

with open('conf/draping.json','r') as f:
  config = json.load(f)

def save_config(config, file):
  with open(file,'w') as f:
    json.dump(config, f)

save_config(config, out_path+'/conf.json')

torch.set_num_threads(8)
spf = config['frame_steps']
frame_time = config['frame_time']
scalev=1

pre_steps = 10
#steps=30
epochs=10

seed_point_index = 292
#handles = [302, 198, 2, 23]
boundary = [  2,   1,  68,  74,  77,  79,  81,  83,  85,  87,  89, 198,  65, 
         63,  61,  59,  57,  55,  53,  51,  49,  47,  45,  43,  41,  39,
         37,  35,  33,  31,  29,  27,  25,  24,  23,  21,  19,  17,  15,
         13,  11,   9,   7,   4,   3, 302,  92,  91,  95,  98,  97, 101,
        103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127,
        129, 131, 133, 139, 141,   0]

def reset_sim(sim, epoch):
  if epoch < epochs:
    arcsim.init_physics(out_path+'/conf.json', out_path+'/out%d'%epoch,False)
  else:
    arcsim.init_physics(out_path+'/conf.json',out_path+'/out',False)

def run_sim(steps,sim,param_v):

  loss = 0.

  print("step")
  for step in range(pre_steps + steps):
    print(step)

    if step > pre_steps:
      for i in range(len(handles)):
        sim.cloths[0].mesh.nodes[handles[i]].v += param_v[step-pre_steps,i] * spf
        print(sim.cloths[0].mesh.nodes[handles[i]].x)

    # from IPython.core.debugger import set_trace
    # set_trace()

    #print(sim.obstacles[0].curr_state_mesh.nodes[seed_point_index].x - sim.cloths[0].mesh.nodes[seed_point_index].x)
    
    loss_idxs = [*boundary]

    if step > pre_steps:
      for i in loss_idxs:
        loss += torch.norm(sim.obstacles[0].curr_state_mesh.nodes[i].x - sim.cloths[0].mesh.nodes[i].x, p=1) #* (1 + ((step-pre_steps)/steps))

    arcsim.sim_step()


  v_limit = 0.5 #was 0.25
  stiffness = 2
  weight = 2/30 * steps * 1/7 * len(loss_idxs)
  v_constraint = torch.sum(weight / (1.0+torch.exp(stiffness*(1.0 - (torch.abs(param_v)/v_limit)))))

  avg = torch.mean(param_v,dim=1,keepdim=True)
  reg = (torch.norm(param_v-avg, dim=2)**2).mean()

  total_loss = loss + reg + v_constraint

  return total_loss

def do_train(cur_step,optimizer,scheduler,sim,param_v):
  epoch = 0
  while True:
    reset_sim(sim, epoch)
    st = time.time()
    loss = run_sim(steps, sim, param_v)
    en0 = time.time()
    optimizer.zero_grad()

    loss.backward(retain_graph=True)

    en1 = time.time()
    print("=======================================")
    #print(param_v.data)
    #print(param_v.grad.data)
    f.write('epoch {}:  loss={} \n'.format(epoch,  loss.data))
    print('epoch {}:  loss={} \n'.format(epoch, loss.data))

    print('forward time={}'.format(en0-st))
    print('backward time={}'.format(en1-en0))

    param_v.grad.data.clamp_(-25,25)

    optimizer.step()

    arcsim.delete_mesh(sim.cloths[0].mesh)

    scheduler.step(loss.item())

    if epoch>=epochs:
      break
    epoch = epoch + 1

with open(out_path+('/log%s.txt'%timestamp),'w',buffering=1) as f:
  tot_step = 1
  sim=arcsim.get_sim()
  # reset_sim(sim)

  #param_v = torch.zeros([steps, 4,3],dtype=torch.float64, requires_grad=True)
  param_v = torch.tensor(trajectories/frame_time,dtype=torch.float64, requires_grad=True)

  optimizer = torch.optim.Adam([param_v],lr=0.025)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=0, verbose=True)

  for cur_step in range(tot_step):
    do_train(cur_step,optimizer,scheduler,sim,param_v)

print("done")


step
0
1
2
3
4
5
6
7
8
9
10
11
tensor([-1.0238,  1.0755,  0.2785], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([ 0.5603, -1.7197,  0.2836], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([-0.3985, -2.1977,  0.2902], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([1.0238, 1.3728, 0.2820], dtype=torch.float64, grad_fn=<AddBackward0>)
12
tensor([-1.0184,  1.0752,  0.2506], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([ 0.5583, -1.7184,  0.2589], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([-0.3877, -2.1940,  0.2487], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([1.0126, 1.3720, 0.2419], dtype=torch.float64, grad_fn=<AddBackward0>)
13
tensor([-1.0136,  1.0752,  0.2231], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([ 0.5562, -1.7170,  0.2341], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([-0.3771, -2.1909,  0.2075], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([1.0022,

KeyboardInterrupt: ignored

In [ ]:
%cd /content/diffsim/pysim

/content/diffsim/pysim


In [ ]:
%%writefile visualize.py
import torch
import arcsim

with torch.autograd.profiler.profile() as prof:
	  arcsim.msim(4,['arcsim','replay','/content/default_out/out', '/content/default_out/out'])
print(prof)

Overwriting visualize.py


In [ ]:
!xvfb-run -s "-screen 0 1280x720x24 -ac +extension GLX +render -noreset" python visualize.py

main
4
arcsim replay /content/default_out/out
reply 1 
reply 2 
reply 3 
reply 4 
reply 5 
reply 6 
/content/default_out/out/0000_
/content/default_out/out/0000_rig
reply 7 
/content/default_out/out/0000.bmp
/content/default_out/out/0001_
/content/default_out/out/0001_rig
/content/default_out/out/0001.bmp
/content/default_out/out/0002_
/content/default_out/out/0002_rig
/content/default_out/out/0002.bmp
/content/default_out/out/0003_
/content/default_out/out/0003_rig
/content/default_out/out/0003.bmp
/content/default_out/out/0004_
/content/default_out/out/0004_rig
/content/default_out/out/0004.bmp
/content/default_out/out/0005_
/content/default_out/out/0005_rig
/content/default_out/out/0005.bmp
/content/default_out/out/0006_
/content/default_out/out/0006_rig
/content/default_out/out/0006.bmp
/content/default_out/out/0007_
/content/default_out/out/0007_rig
/content/default_out/out/0007.bmp
/content/default_out/out/0008_
/content/default_out/out/0008_rig
/content/default_out/out/0008.bmp


In [ ]:
import pathlib
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

frames = []
for bmp_file in sorted(list(pathlib.Path('/content/default_out/out').glob('*.bmp'))):
    frames.append(plt.imread(bmp_file))
    #print(bmp_file)
    

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

fig = plt.figure()

ims = []
for i in range(len(frames)):
    im = plt.imshow(frames[i], animated=True)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)

rc('animation', html='jshtml')
ani

In [ ]:
HTML(ani.to_html5_video())

In [ ]:
print(param_v)

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[-2.7177e-02, -6.9709e-02, -1.0069e-01],
         [ 3.8210e-02, -2.6388e-02, -1.2819e-01],
         [ 2.3521e-01, -1.5309e-03, -9.2511e-02],
         [-1.0704e-01, -7.9604e-02, -1.7047e-01]],

        [[ 1.2869e-01, -1.2539e-01, -2.0540e-01],
         [-4.0851e-02, -2.4771e-02, -5.2898e-02],
         [ 1.6915e-01,  2.0242e-01, -5.7146e-02],
         [-2.1052e-01, -9.1641e-02, -2.1956e-01]],

        [[ 5.3117e-02, -6.1831e-02, -1.4591e-01],
         [-7.0635e-02,  7.6858e-02, -1.2274e-01],
         [ 3.8750e-02,  1.5665e-01, -1.4255e-01],
         [-1.1924e-01, -4.5122e-03, -1.2790e-01]],

        [[ 1.1208e-01, -8.2489e-02, -2.0415e-01],
         [-6.0018e-02,  1.1944e-01, -1.2472e-01],
         [ 6.8102e-02,  2.5328e-02, -1.8933e-01],
         [-1.0932e-01, -5.7377e-02, -2.091

In [ ]:
param_v.grad

tensor([[[  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000]],

        [[-35.0000,  35.0000,  35.0000],
         [ 35.0000,  35.0000,  35.0000],
         [ 35.0000,  35.0000, -35.0000],
         [ 30.2527,  35.0000, -35.0000]],

        [[-35.0000,  35.0000,  35.0000],
         [ 35.0000, -35.0000,  35.0000],
         [ 35.0000, -35.0000, -35.0000],
         [ 35.0000,  35.0000,  35.0000]],

        [[-35.0000,  35.0000,  35.0000],
         [ 35.0000, -35.0000,  35.0000],
         [-35.0000,  35.0000, -35.0000],
         [-35.0000,  35.0000, -35.0000]],

        [[-35.0000,  35.0000,  35.0000],
         [ 35.0000, -35.0000,  35.0000],
         [ 35.0000,  35.0000, -35.0000],
         [-35.0000,  35.0000,  35.0000]],

        [[-35.0000,  35.0000,  35.0000],
         [ 35.0000, -35.0000,  35.0000],
         [ 35.0000,  35.0000,  35.0000],
         [ -6.7152,  28.2333, -35.0000]],

    

In [ ]:
ani.save('diffdrape.mp4')

In [ ]:
torch.save(param_v, 'trajectory.pt')